# Wrangling of Existing US Accident and Weather Dataset
In this initial step I'll fill or remove null values, add useful columns, extract outliers and make corrections.

This dataset was found at https://osu.app.box.com/v/us-accidents-dec19 and unzipped with 7-Zip

In [ ]:
import numpy as np
import pandas as pd
import datetime as dt

In [2]:
df = pd.read_csv('C:/data/US_Accidents_Dec19.csv', parse_dates = ['Start_Time', 'End_Time'])  

In [3]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2974335 entries, 0 to 2974334
Data columns (total 49 columns):
 #   Column                 Dtype         
---  ------                 -----         
 0   ID                     object        
 1   Source                 object        
 2   TMC                    float64       
 3   Severity               int64         
 4   Start_Time             datetime64[ns]
 5   End_Time               datetime64[ns]
 6   Start_Lat              float64       
 7   Start_Lng              float64       
 8   End_Lat                float64       
 9   End_Lng                float64       
 10  Distance(mi)           float64       
 11  Description            object        
 12  Number                 float64       
 13  Street                 object        
 14  Side                   object        
 15  City                   object        
 16  County                 object        
 17  State                  object        
 18  Zipcode               

In [35]:
orig_count = df.ID.count()

In [37]:
# % null
round((df.isna().sum() / orig_count * 100),1)

ID                        0.0
Source                    0.0
TMC                      24.5
Severity                  0.0
Start_Time                0.0
End_Time                  0.0
Start_Lat                 0.0
Start_Lng                 0.0
End_Lat                  75.5
End_Lng                  75.5
Distance(mi)              0.0
Description               0.0
Number                   64.5
Street                    0.0
Side                      0.0
City                      0.0
County                    0.0
State                     0.0
Zipcode                   0.0
Country                   0.0
Timezone                  0.1
Airport_Code              0.2
Weather_Timestamp         1.2
Temperature(F)            1.9
Wind_Chill(F)            62.3
Humidity(%)               2.0
Pressure(in)              1.6
Visibility(mi)            2.2
Wind_Direction            1.5
Wind_Speed(mph)          14.8
Precipitation(in)        67.2
Weather_Condition         2.2
Amenity                   0.0
Bump      

In [5]:
# Create a time of traffic duration
df['Duration'] = df.apply(lambda x: x['End_Time'] - x['Start_Time'], axis = 1)

# Convert to minutes
df['Duration(m)'] = df['Duration']/np.timedelta64(1,'m')

In [18]:
# Swap Start Time and End Time when Duration is negative
for i in df.index:
    if df.loc[i, 'Duration(m)'] < 0:
        t = df.loc[i, 'Start_Time']
        df.loc[i, 'Start_Time'] = df.loc[i, 'End_Time']
        df.loc[i, 'End_Time'] = t
        df.loc[i, 'Duration(m)'] = abs(df.loc[i, 'Duration(m)'])
        df.loc[i, 'Duration'] = df.loc[i, 'End_Time'] - df.loc[i, 'Start_Time']
    else:
        continue

In [26]:
# Add day of week column
df['weekday'] = df['Start_Time'].apply(lambda x: dt.datetime.strftime(x, '%a'))

In [27]:
# Add hour column
df['hour'] = df['Start_Time'].apply(lambda x: dt.datetime.strftime(x, '%H'))

In [28]:
# End_Lat & _Lng are 75.5% null
# Can records have Distance without End coordinates?  Yes
df[(df['End_Lat'].isnull() == True) & (df['Distance(mi)'] > 0.1)].shape

(132433, 53)

In [29]:
# Can Distance be zero and End_Lat not null?  Yes
df[(df['End_Lat'].notnull() == True) & (df['Distance(mi)'] == 0)].shape
# End_Lat and End_Lng are not required.

(152280, 53)

In [ ]:
# Street Number is 64.5% null and not required, considering highways and ramps, etc., do not have a street address.

In [30]:
# TMC is 24.5% null, while 63.5% of available points are in one category, and likely not beneficial.
# % in TMC categories
df.TMC.value_counts() / orig_count * 100

201.0    63.544860
241.0     7.365377
245.0     1.207631
229.0     0.761616
203.0     0.564563
222.0     0.422010
244.0     0.375311
406.0     0.318290
246.0     0.213493
202.0     0.203306
343.0     0.195909
247.0     0.155194
236.0     0.071142
206.0     0.038899
248.0     0.034024
339.0     0.026023
341.0     0.016945
336.0     0.002723
200.0     0.002219
239.0     0.001816
351.0     0.000202
Name: TMC, dtype: float64

In [31]:
# Exclude End_Lat, End_Lng, Number (street), and TMC columns
df2 = df.drop(['End_Lat', 'End_Lng', 'Number', 'TMC'], axis = 1)
df2.shape

(2974335, 49)

In [33]:
# Drop remaining NaN records
df_NoNaN = df2.dropna()
df_NoNaN.shape

(779721, 49)

In [34]:
# Current subset with no NaN is 26% of original file
round( df_NoNaN.ID.count() /  orig_count *100,1)

26.2

In [ ]:
# Original file was over 60% null in Precipitation(in) and Wind_Chill(F)
# Below is an effort to salvage a greater proportion of records:
# Sort on month and zipcode to group similar weather together and replace NaNs with interpolate.

In [38]:
# Zipcode is null in only 0.03% of records, remove null zipcode rows
df2 = df2[df2['Zipcode'].notna()]

In [39]:
# Shorten zipcode for larger areas
df2['Zip2'] = df2['Zipcode'].str[:2]

In [40]:
# Extract month
df2['Month'] = df2['Start_Time'].dt.month

In [41]:
# Sort by Month and Zip code to group similar weather
df2.sort_values(by = ['Month', 'Zip2'], inplace = True)

In [42]:
# Interpolate to fill missing records in Precipitation, Wind_Chill and Wind_Speed
cols = ['Precipitation(in)', 'Wind_Chill(F)', 'Wind_Speed(mph)']
df2[cols] = df2[cols].interpolate(method ='linear', axis = 0, limit_direction = 'both') 

In [44]:
# Delete remaining null values in columns with 1-2% missing
df2 = df2.dropna()

In [45]:
# Retained 97% of original records
round((df2.ID.count() / orig_count * 100), 1)

96.9

In [46]:
# remove outliers
from scipy import stats

def drop_numerical_outliers(df, z_thresh=3):
    # Constrains will contain `True` or `False` depending on if it is a value below the threshold.
    constrains = df.select_dtypes(include='float64') \
        .apply(lambda x: np.abs(stats.zscore(x)) < z_thresh, result_type='reduce') \
        .all(axis=1)
    # Drop (inplace) values set to be rejected
    df.drop(df.index[~constrains], inplace=True)

In [47]:
drop_numerical_outliers(df2)

In [48]:
# Removing outliers leaves 91% of original record count
round((df2.ID.count() / orig_count * 100), 1)

91.4

In [49]:
# Output wrangled data
df2.to_csv('C:/data/Accidents.csv')